In [12]:
import torch
import json
from tqdm import tqdm
import os
from src.train_and_evaluate import *
from src.models import *
import time
import torch.optim
from src.expressions_transfer import *
import numpy as np

def read_json(path):
    with open(path,'r') as f:
        file = json.load(f)
    return file

def write_json(path,file):
    with open(path,'w') as f:
        json.dump(file,f)

In [60]:
sftarget_path = 'data/node_seq_323.pt'
node = torch.load(sftarget_path)
node_idx_path = 'data/idx_seq_323.json'
node_idx = read_json(node_idx_path)

In [62]:
print(len(node))
print(node[0][0].shape)
print(len(node_idx))
print(node_idx[0])

331
torch.Size([64, 15])
331
[63, 53, 56, 30, 7, 45, 58, 0, 20, 32, 26, 46, 21, 61, 11, 16, 29, 31, 44, 19, 52, 1, 48, 36, 59, 2, 14, 17, 9, 55, 5, 8, 51, 10, 28, 62, 27, 54, 33, 37, 40, 15, 25, 35, 38, 3, 57, 34, 39, 41, 42, 6, 23, 4, 13, 49, 50, 43, 18, 60, 12, 22, 47, 24]


In [63]:
node_t = copy.copy(node[0])
node_t = torch.stack(node_t,dim=1)
print(node_t.shape)
print(node_t[0].shape)
print(node_t[0])


torch.Size([64, 15, 15])
torch.Size([15, 15])
tensor([[ 3.0673e-03,  1.3570e-01,  3.3574e-03,  1.2970e-02,  7.2053e-02,
          1.9219e-02, -4.9919e-01, -1.0012e-02,  1.2002e-03,  9.4832e-02,
          1.2632e-02,  3.8763e-02, -1.0000e+12, -1.0000e+12, -1.0000e+12],
        [ 6.9916e-02,  2.1949e-02,  1.2365e-03, -9.4975e-02,  9.2628e-02,
         -1.5463e-01,  1.9664e-01,  6.0962e-02, -4.1057e-02,  5.6526e-02,
          1.4279e-01,  7.7104e-02, -1.0000e+12, -1.0000e+12, -1.0000e+12],
        [ 4.7307e-03,  7.1127e-02,  1.5039e-03,  4.6854e-03,  7.6305e-02,
         -1.6820e-01, -2.5594e-01,  8.0312e-02,  1.1215e-01, -2.3642e-02,
          1.3829e-01,  8.8516e-02, -1.0000e+12, -1.0000e+12, -1.0000e+12],
        [-4.2035e-02, -1.7013e-02,  3.0909e-02, -8.0863e-02,  1.5259e-01,
          6.3022e-02, -2.7987e-02,  1.3243e-01,  5.1127e-03, -1.3532e-02,
          7.0612e-02,  3.3528e-02, -1.0000e+12, -1.0000e+12, -1.0000e+12],
        [ 8.9785e-02,  1.3559e-01,  9.6120e-02, -5.1938e-02,  

In [169]:
def pad_label(seq, max_length):
    seq_new = copy.copy(seq)
    seq_len = len(seq_new)
    if seq_len < max_length:
        pad_len = max_length - seq_len
        seq_new += [0.0 for _ in range(pad_len)]
    else:
        while len(seq_new) > max_length:
            seq_new.pop()
    #seq += [0.0 for _ in range(max_length - seq_len)]
    #seq_new = torch.from_numpy(np.array(seq_new))
    seq_new = [float(item) for item in seq_new]
    #print(seq_new)
    
    return seq_new

def pad_group(seq_list, seq_length, max_length):
    seq_new = []
    current_seq_length = len(seq_list.split(1,dim=0))
    for seq in seq_list.split(1,dim=0):
        seq = seq.squeeze().detach().numpy().tolist()
        seq_new.append(pad_label(seq, max_length))
    if current_seq_length < seq_length:
        pad_len = seq_length - current_seq_length
        for i in range(pad_len):
            seq_new.append([0 for i in range(max_length)])
    elif current_seq_length > seq_length:
        while len(seq_new) > seq_length:
            seq_new.pop()
    #print(type(seq_new))
    #print(type(seq_new[1]))
    #print(seq_new)
    #print(seq_new[1])
    #print(seq_new[-1])
    np_seq_new = np.array(seq_new)
    #print(type(np_seq_new))
    #print(np_seq_new)
    seq_new = torch.from_numpy(np.array(seq_new))
    return seq_new

def pad_tensor(seq_tensor, seq_length, max_length):
    seq_new = []
    for seq_matrix in seq_tensor.split(1,dim=0):
        seq_t = pad_group(seq_matrix.squeeze(), seq_length, max_length)
        seq_new.append(seq_t)
    return torch.stack(seq_new,dim=0)

def get_softtarget_list(node):
    mask_list = []
    node_copy = copy.copy(node)
    for node_t in node_copy:
        node_t = torch.stack(node_t,dim=1)
        for i in range(node_t.size(0)):
            mask = node_t[1].ne(-1.0000e12).float().cpu()
            node_t_cpu = node_t[i].cpu()
            mask_list.append(mask*node_t_cpu)
    return mask_list

def get_softtarget_dic(mask_list, node_idx):
    node_idx_whole = []
    for item in node_idx:
        node_idx_whole.extend(item)
    return dict(zip(node_idx_whole,mask_list))

def get_softtarget_vector(batch_idx,sf_dic):
    for idx in batch_idx:
        current_item = sf_dic[idx]
        
    return [sf_dic[idx] for idx in batch_idx]

sf_dic = get_softtarget_dic(get_softtarget_list(node), node_idx)
# -----------------------------------
#max_length = 18
batch_idx = node_idx[0]
batch_sf = get_softtarget_vector(batch_idx,sf_dic)
test_batch_sf = torch.stack(batch_sf,dim=0)

In [171]:
test_tensor = pad_tensor(test_batch_sf, 20, 18)
print(test_batch_sf.shape)
print(test_tensor.shape)

torch.Size([64, 15, 15])
torch.Size([64, 20, 18])


In [165]:
test_group = pad_group(test_batch_sf[0], 18, 12)

In [166]:
print(test_group.shape)

torch.Size([18, 12])


In [167]:
print(test_group)

tensor([[ 3.0673e-03,  1.3570e-01,  3.3574e-03,  1.2970e-02,  7.2053e-02,
          1.9219e-02, -4.9919e-01, -1.0012e-02,  1.2002e-03,  9.4832e-02,
          1.2632e-02,  0.0000e+00],
        [ 6.9916e-02,  2.1949e-02,  1.2365e-03, -9.4975e-02,  9.2628e-02,
         -1.5463e-01,  1.9664e-01,  6.0962e-02, -4.1057e-02,  5.6526e-02,
          1.4279e-01,  0.0000e+00],
        [ 4.7307e-03,  7.1127e-02,  1.5039e-03,  4.6854e-03,  7.6305e-02,
         -1.6820e-01, -2.5594e-01,  8.0312e-02,  1.1215e-01, -2.3642e-02,
          1.3829e-01,  0.0000e+00],
        [-4.2035e-02, -1.7013e-02,  3.0909e-02, -8.0863e-02,  1.5259e-01,
          6.3022e-02, -2.7987e-02,  1.3243e-01,  5.1127e-03, -1.3532e-02,
          7.0612e-02,  0.0000e+00],
        [ 8.9785e-02,  1.3559e-01,  9.6120e-02, -5.1938e-02,  2.3288e-02,
         -4.1538e-01,  9.3976e-02,  8.7603e-02,  5.1946e-02,  1.5023e-01,
          1.2160e-01,  0.0000e+00],
        [ 1.3529e-02, -3.7193e-02,  1.1076e-01, -4.8971e-02,  5.2443e-02,
      

In [158]:
test_tensor = pad_tensor(test_batch_sf, 20, 18)

In [159]:
print(test_tensor.shape)

torch.Size([64, 20, 18])


In [111]:
print(type(test_group))

<class 'list'>


In [108]:
print(len(test_group))

18


In [110]:
print(torch.stack(test_group,dim=0).shape)

TypeError: expected Tensor as element 0 in argument 0, but got list

In [98]:
seq_list = list(range(15))
print(seq_list)
seq_list.pop()
print(seq_list)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]


In [89]:
def get_softtarget_vector(batch_idx,sf_dic):
    return [sf_dic[idx] for idx in batch_idx]

batch_idx = node_idx[0]
batch_sf = get_softtarget_vector(batch_idx,sf_dic)
print(len(batch_sf))

64


In [90]:
test_batch_sf = torch.stack(batch_sf,dim=0)
print(test_batch_sf.shape)

torch.Size([64, 15, 15])


In [99]:
print(test_batch_sf[0].shape)


torch.Size([15, 15])


In [93]:
print(len(test_batch_sf.split(1,dim=0)))
for item in test_batch_sf.split(1,dim=0):
    print(item.shape)

64
torch.Size([1, 15, 15])
torch.Size([1, 15, 15])
torch.Size([1, 15, 15])
torch.Size([1, 15, 15])
torch.Size([1, 15, 15])
torch.Size([1, 15, 15])
torch.Size([1, 15, 15])
torch.Size([1, 15, 15])
torch.Size([1, 15, 15])
torch.Size([1, 15, 15])
torch.Size([1, 15, 15])
torch.Size([1, 15, 15])
torch.Size([1, 15, 15])
torch.Size([1, 15, 15])
torch.Size([1, 15, 15])
torch.Size([1, 15, 15])
torch.Size([1, 15, 15])
torch.Size([1, 15, 15])
torch.Size([1, 15, 15])
torch.Size([1, 15, 15])
torch.Size([1, 15, 15])
torch.Size([1, 15, 15])
torch.Size([1, 15, 15])
torch.Size([1, 15, 15])
torch.Size([1, 15, 15])
torch.Size([1, 15, 15])
torch.Size([1, 15, 15])
torch.Size([1, 15, 15])
torch.Size([1, 15, 15])
torch.Size([1, 15, 15])
torch.Size([1, 15, 15])
torch.Size([1, 15, 15])
torch.Size([1, 15, 15])
torch.Size([1, 15, 15])
torch.Size([1, 15, 15])
torch.Size([1, 15, 15])
torch.Size([1, 15, 15])
torch.Size([1, 15, 15])
torch.Size([1, 15, 15])
torch.Size([1, 15, 15])
torch.Size([1, 15, 15])
torch.Size([1

In [80]:
sf_dic.keys()

dict_keys([63, 53, 56, 30, 7, 45, 58, 0, 20, 32, 26, 46, 21, 61, 11, 16, 29, 31, 44, 19, 52, 1, 48, 36, 59, 2, 14, 17, 9, 55, 5, 8, 51, 10, 28, 62, 27, 54, 33, 37, 40, 15, 25, 35, 38, 3, 57, 34, 39, 41, 42, 6, 23, 4, 13, 49, 50, 43, 18, 60, 12, 22, 47, 24, 72, 79, 127, 91, 104, 65, 113, 76, 84, 116, 114, 89, 109, 124, 70, 73, 87, 112, 117, 67, 107, 122, 74, 80, 88, 96, 68, 75, 99, 92, 94, 106, 126, 98, 101, 103, 123, 105, 118, 71, 77, 90, 108, 81, 64, 83, 111, 97, 119, 95, 78, 110, 121, 66, 69, 82, 102, 125, 85, 115, 86, 100, 120, 93, 143, 138, 189, 188, 165, 139, 160, 128, 152, 156, 178, 184, 166, 162, 191, 161, 175, 179, 180, 187, 153, 177, 140, 185, 144, 145, 176, 157, 173, 150, 155, 158, 171, 131, 181, 129, 134, 142, 186, 132, 135, 167, 133, 137, 169, 149, 182, 130, 148, 163, 141, 151, 172, 164, 174, 170, 183, 146, 168, 147, 136, 159, 190, 154, 223, 228, 219, 249, 192, 207, 209, 215, 225, 208, 198, 222, 203, 220, 253, 201, 205, 244, 254, 227, 202, 206, 226, 237, 194, 199, 248, 250,

In [64]:
#torch.eq(node_t[1],-1.0000e12)
mask = node_t[1].ne(-1.0000e12).float().cpu()
node_t_cpu = node_t[1].cpu()
print((mask*node_t_cpu).split(1,dim=0))
mask_node_list = (mask*node_t_cpu).split(1,dim=0)
print(mask_node_list[0])
print(mask_node_list[0].size(1))

(tensor([[-0.0900,  0.0223,  0.0907, -0.1222,  0.1475, -0.2086,  0.0630,  0.1218,
          0.0538, -0.0431,  0.0619, -0.0000, -0.0000, -0.0000, -0.0000]],
       grad_fn=<SplitBackward>), tensor([[ 0.0152,  0.0774,  0.0826, -0.0194,  0.0862, -0.1095, -0.1408,  0.0720,
          0.1337, -0.1310,  0.1293, -0.0000, -0.0000, -0.0000, -0.0000]],
       grad_fn=<SplitBackward>), tensor([[-0.0463,  0.0618,  0.0523, -0.0220,  0.2059, -0.1592, -0.0854, -0.0160,
          0.0371, -0.1118, -0.0203, -0.0000, -0.0000, -0.0000, -0.0000]],
       grad_fn=<SplitBackward>), tensor([[ 0.1442,  0.0306, -0.0170, -0.3729,  0.2853, -0.1706, -0.1358,  0.1135,
          0.1181, -0.0541,  0.1121, -0.0000, -0.0000, -0.0000, -0.0000]],
       grad_fn=<SplitBackward>), tensor([[-0.0363,  0.0518,  0.1483, -0.0434,  0.1258, -0.3904, -0.2013, -0.0273,
          0.1165, -0.0360,  0.1709, -0.0000, -0.0000, -0.0000, -0.0000]],
       grad_fn=<SplitBackward>), tensor([[-0.0261, -0.0672,  0.0984, -0.0447,  0.1464, -0.34

In [65]:
# Pad a with the PAD symbol
def pad_label(seq, max_length):
    seq_len = len(seq)
    if seq_len < max_length:
        pad_len = max_length - seq_len
        seq += [0.0 for _ in range(pad_len)]
    else:
        while len(seq) > max_length:
            seq.pop()
    #seq += [0.0 for _ in range(max_length - seq_len)]
    return seq

In [67]:
seq_test = mask_node_list[0].squeeze().detach().numpy().tolist()
pad_test = pad_label(seq_test,12)
print(pad_test)
print(len(pad_test))

[-0.09002731740474701, 0.022344272583723068, 0.09073850512504578, -0.12224400788545609, 0.14753742516040802, -0.20862969756126404, 0.06301473826169968, 0.12179248034954071, 0.053831566125154495, -0.04310405254364014, 0.06192971020936966, -0.0]
12


In [ ]:
def 
mask = node_t[1].ne(-1.0000e12).float().cpu()
node_t_cpu = node_t[1].cpu()
print((mask*node_t_cpu).split(1,dim=0))
mask_node_list = (mask*node_t_cpu).split(1,dim=0)

In [13]:
batch_size = 64
embedding_size = 128
hidden_size = 512
n_epochs = 80
learning_rate = 1e-3
weight_decay = 1e-5
beam_size = 5
n_layers = 2
ori_path = './data/'
prefix = '23k_processed.json'

def load_raw_data(filename):  # load the json data to list(dict()) for MATH 23K
    print("Reading lines...")
    f = open(filename, encoding="utf-8")
    js = ""
    data = []
    for i, s in enumerate(f):
        js += s
        i += 1
        if i % 7 == 0:  # every 7 line is a json
            data_d = json.loads(js)
            if "千米/小时" in data_d["equation"]:
                data_d["equation"] = data_d["equation"][:-5]
            data.append(data_d)
            js = ""

    return data

def get_train_test_fold(ori_path,prefix,data,pairs,group):
    mode_train = 'train'
    mode_valid = 'valid'
    mode_test = 'test'
    train_path = ori_path + mode_train + prefix
    valid_path = ori_path + mode_valid + prefix
    test_path = ori_path + mode_test + prefix
    train = read_json(train_path)
    train_id = [item['id'] for item in train]
    valid = read_json(valid_path)
    valid_id = [item['id'] for item in valid]
    test = read_json(test_path)
    test_id = [item['id'] for item in test]
    train_fold = []
    valid_fold = []
    test_fold = []
    for item,pair,g in zip(data, pairs, group):
        pair = list(pair)
        pair.append(g['group_num'])
        pair = tuple(pair)
        if item['id'] in train_id:
            train_fold.append(pair)
        elif item['id'] in test_id:
            test_fold.append(pair)
        else:
            valid_fold.append(pair)
    return train_fold, test_fold, valid_fold

def change_num(num):
    new_num = []
    for item in num:
        if '/' in item:
            new_str = item.split(')')[0]
            new_str = new_str.split('(')[1]
            a = float(new_str.split('/')[0])
            b = float(new_str.split('/')[1])
            value = a/b
            new_num.append(value)
        elif '%' in item:
            value = float(item[0:-1])/100
            new_num.append(value)
        else:
            new_num.append(float(item))
    return new_num


data = load_raw_data("data/Math_23K.json")
group_data = read_json("data/Math_23K_processed.json")

data = load_raw_data("data/Math_23K.json")

pairs, generate_nums, copy_nums = transfer_num(data)

temp_pairs = []
for p in pairs:
    temp_pairs.append((p[0], from_infix_to_prefix(p[1]), p[2], p[3]))
pairs = temp_pairs

train_fold, test_fold, valid_fold = get_train_test_fold(ori_path,prefix,data,pairs,group_data)

best_acc_fold = []

pairs_tested = test_fold
pairs_trained = train_fold

Reading lines...
Reading lines...
Transfer numbers...


In [15]:
print(pairs_trained[0])

(['镇海', '雅乐', '学校', '二年级', '的', '小朋友', '到', '一条', '小路', '的', '一边', '植树', '．', '小朋友', '们', '每隔', 'NUM', '米', '种', '一棵树', '（', '马路', '两头', '都', '种', '了', '树', '）', '，', '最后', '发现', '一共', '种', '了', 'NUM', '棵', '，', '这', '条', '小路', '长', '多少', '米', '．'], ['*', '-', 'N1', '1', 'N0'], ['2', '11'], [16, 34], [15, 16, 17, 32, 33, 34, 39, 40, 41])


In [172]:
loss_test_tensor = torch.tensor([[[1,2,3,4],[1,2,3,4],[1,2,3,4]],[[1,2,3,4],[1,2,3,4],[1,2,3,4]]])

In [174]:
print(loss_test_tensor.shape)
print(loss_test_tensor)

torch.Size([2, 3, 4])
tensor([[[1, 2, 3, 4],
         [1, 2, 3, 4],
         [1, 2, 3, 4]],

        [[1, 2, 3, 4],
         [1, 2, 3, 4],
         [1, 2, 3, 4]]])


In [175]:
print(loss_test_tensor[:,0,:].shape)
print(loss_test_tensor[:,0,:])

torch.Size([2, 4])
tensor([[1, 2, 3, 4],
        [1, 2, 3, 4]])


In [176]:
for item in loss_test_tensor.split(1,dim=1):
    print(item.shape)
    print(item)

torch.Size([2, 1, 4])
tensor([[[1, 2, 3, 4]],

        [[1, 2, 3, 4]]])
torch.Size([2, 1, 4])
tensor([[[1, 2, 3, 4]],

        [[1, 2, 3, 4]]])
torch.Size([2, 1, 4])
tensor([[[1, 2, 3, 4]],

        [[1, 2, 3, 4]]])


In [177]:
print(torch.zeros(loss_test_tensor.shape))
print(torch.zeros(loss_test_tensor.shape).shape)

tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])
torch.Size([2, 3, 4])


In [180]:
tensor_list = [item.squeeze() for item in loss_test_tensor.split(1,dim=0)]

In [195]:
tensor_list = [torch.zeros(64),torch.zeros(64),torch.zeros(64)]
tensor_list = torch.stack(tensor_list,dim=0)
tensor_list = tensor_list.transpose(1,0)
print(tensor_list.shape)

torch.Size([64, 3])


In [197]:
print(tensor_list.sum(dim=1).shape)

torch.Size([64])


In [182]:
print(tensor_list)

tensor([[[1, 2, 3, 4],
         [1, 2, 3, 4],
         [1, 2, 3, 4]],

        [[1, 2, 3, 4],
         [1, 2, 3, 4],
         [1, 2, 3, 4]]])
